## ** 앙상블 학습과 랜덤 포레스트 연습 문제**
___
- 출처 : 핸즈온 머신러닝 Ch07 앙상블 학습과 랜덤 포레스트 연습문제 2, 7, 8, 9번
- 이론적 지식을 묻는 문제의 경우 텍스트 셀을 추가하여 정답을 적어주세요.

In [1]:
# import libraries
import numpy as np

### **1. 직접 투표와 간접 투표 분류기 사이의 차이점은 무엇일까요?**
___

>직접 투표  <br>
: 각 분류기의 예측을 모아서 가장 많이 선택된 클래스를 예측 결과로 함

>간접투표 <br>
: 개별 분류기의 예측을 평균 내어 확률이 가장 높은 클래스를 예측 결과로 함 

### **2. 그레디언트 부스팅 앙상블이 훈련 데이터에 과대 적합되었다면 학습률을 어떻게 해야 할까요?**
___

> 과대적합이 발생한 경우, 학습률(learning rate)을 줄이는 것이 일반적인 방법 중 하나.
>> 학습률을 줄이면 각 트리가 데이터에 대해 더 조심스럽게 학습하게 되므로 모델이 더욱 부드럽고 일반화되도록 도와줍니다.

### **3. [실습] 다음 지시에 따라 투표 기반 분류 모델을 만들어 보세요**
___

#### **STEP 1. MNIST 데이터를 불러들이고, 훈련, 검증, 테스트 데이터로 나누세요.**

In [2]:
# import MNIST dataset
from sklearn.datasets import fetch_openml

mnist = fetch_openml('mnist_784', version=1, as_frame = False)
X, y = mnist["data"], mnist["target"]

/Users/ijiseon/opt/anaconda3/lib/python3.9/site-packages/sklearn/datasets/_openml.py:1022: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [5]:
# train/valid/test dataset

from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X,y, test_size=0.3)

In [6]:
X_val, X_test, y_val, y_test = train_test_split(X_val,y_val, test_size=0.1)

####  **STEP 2. 랜덤 포레스트 분류기, 엑스트라 트리 분류기, SVM 분류기, MLP 분류기를 훈련시키세요.**
- 모델 파라미터는 `n_estimators=100`, `random_state=42`로 설정합니다.

In [7]:
# import package
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier

In [8]:
# model fitting

rnd_clf = RandomForestClassifier(n_estimators=100,random_state=42, n_jobs=-1)
rnd_clf.fit(X_train,y_train)

et_clf = ExtraTreesClassifier(n_estimators=500, max_leaf_nodes=16, n_jobs=-1)
et_clf.fit(X_train,y_train)


svm_clf = LinearSVC(random_state=42)
svm_clf.fit(X_train, y_train)


mlp_clf = MLPClassifier(random_state=42)
mlp_clf.fit(X_train, y_train)

/Users/ijiseon/opt/anaconda3/lib/python3.9/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/Users/ijiseon/opt/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


MLPClassifier(random_state=42)

####  **STEP 3-1. 앞에서 훈련시킨 각 모델을 직접 투표 방법을 사용해 앙상블로 연결하고 훈련시킨 후, `score()`메서드를 이용하여 검증 데이터셋에서의 성능을 평가해보세요.**

In [10]:
from sklearn.ensemble import VotingClassifier

voting_clf = VotingClassifier(
    estimators = [("et",et_clf ), ("rf", rnd_clf), ("svc", svm_clf), ("mlp", mlp_clf)],
    voting = "hard"
)

voting_clf.fit(X_train,y_train)

/Users/ijiseon/opt/anaconda3/lib/python3.9/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/Users/ijiseon/opt/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


VotingClassifier(estimators=[('et',
                              ExtraTreesClassifier(max_leaf_nodes=16,
                                                   n_estimators=500,
                                                   n_jobs=-1)),
                             ('rf',
                              RandomForestClassifier(n_jobs=-1,
                                                     random_state=42)),
                             ('svc', LinearSVC(random_state=42)),
                             ('mlp', MLPClassifier(random_state=42))])

In [13]:
# model test

from sklearn.metrics import accuracy_score

y_pred = voting_clf.predict(X_val)


print(accuracy_score(y_val,y_pred))

0.9471957671957671


####  **STEP 3-2. 검증 데이터셋에서 각 분류 모델의 성능을 `score()` 메서드를 이용하여 확인해보고, 가장 성능이 낮은 모델을 제거하여 그 결과를 비교해보세요.**
- Hint : 가장 성능이 낮은 모델을 제거할 때 `del`를 활용해보세요

In [14]:
# 각 분류 모델 학습

from sklearn.metrics import accuracy_score

for clf in (et_clf,  rnd_clf, svm_clf, mlp_clf):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_val)
    print(clf.__class__.__name__, accuracy_score(y_val,y_pred))


ExtraTreesClassifier 0.8098412698412698
RandomForestClassifier 0.9688359788359788


/Users/ijiseon/opt/anaconda3/lib/python3.9/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/Users/ijiseon/opt/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


LinearSVC 0.8660846560846561
MLPClassifier 0.9632275132275132


- Q. 어떤 모델의 성능이 가장 낮나요?
- A. ExtraTreesClassifier

In [17]:
# 가장 성능이 낮은 모델 제거

models = {"et": et_clf, "rf": rnd_clf, "svc": svm_clf, "mlp": mlp_clf}

del models["et"]

voting_clf = VotingClassifier(estimators=list(models.items()), voting="hard")


In [18]:
# model fitting

voting_clf.fit(X_train, y_train)

/Users/ijiseon/opt/anaconda3/lib/python3.9/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/Users/ijiseon/opt/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


VotingClassifier(estimators=[('rf',
                              RandomForestClassifier(n_jobs=-1,
                                                     random_state=42)),
                             ('svc', LinearSVC(random_state=42)),
                             ('mlp', MLPClassifier(random_state=42))])

In [19]:
# 모델 제거 후 성능 확인


y_pred = voting_clf.predict(X_val)

print(accuracy_score(y_val,y_pred))

0.9656084656084656


### **4. 다음 단계를 따라 앞에서 훈련시킨 분류 모델들을 이용하여 스태킹 앙상블을 구성해보자.**
___

#### **STEP 1. 3번 문제의 각 분류 모델을 실행해서 검증 세트에서 예측을 만들고, 그 결과로 훈련 세트를 만들어 보세요.**

In [21]:
from sklearn.ensemble import StackingClassifier


meta_estimator = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
stacking_clf = StackingClassifier(estimators=list(models.items()), final_estimator=meta_estimator)
stacking_clf.fit(X_val, y_val)

/Users/ijiseon/opt/anaconda3/lib/python3.9/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/Users/ijiseon/opt/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/ijiseon/opt/anaconda3/lib/python3.9/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/Users/ijiseon/opt/anaconda3/lib/python3.9/site-packages/sklearn/svm/_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/ijiseon/opt/anaconda3/lib/python3.9/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` wi

StackingClassifier(estimators=[('rf',
                                RandomForestClassifier(n_jobs=-1,
                                                       random_state=42)),
                               ('svc', LinearSVC(random_state=42)),
                               ('mlp', MLPClassifier(random_state=42))],
                   final_estimator=RandomForestClassifier(n_jobs=-1,
                                                          random_state=42))

####  **STEP 2. 새로운 훈련 세트를 이용하여 랜덤 포레스트 분류 모델을 학습시켜 보세요.**

- 이 랜덤 포레스트 분류 모델이 바로 블렌더에 해당합니다.

####  **STEP 3. 이제 테스트셋에서 스태킹 앙상블 모델을 평가해보세요.**
- 성능 평가 지표로 **정확도**를 이용하세요.

In [22]:
# 테스트 데이터에 대한 예측
y_pred_stacking = stacking_clf.predict(X_test)

# 스태킹 분류기의 정확도 출력
print("Stacking Classifier Accuracy:", accuracy_score(y_test, y_pred_stacking))

Stacking Classifier Accuracy: 0.9561904761904761
